In [2]:
import os, pydicom, argparse
import nibabel as nib
import numpy as np
import ipywidgets as ipw
import itertools as it

def sort_dcms_by_slice_pos(input_dicom_path,dcm_files,stop_before_pixels=True):
    '''
    Sort DICOMs from an input directory (assumed to contain a single study) according to slice position
    Output: a sorted list of DICOM datasets
    '''
    dcmss=[]
    for idx,dcm in enumerate(dcm_files):
        ds = pydicom.dcmread(os.path.join(input_dicom_path,dcm), stop_before_pixels)
        if idx==0:           
            if 'ImagePositionPatient' in ds: sortTag='ImagePositionPatient'
            elif 'SliceLocation' in ds: sortTag='SliceLocation'
            else: return None
        if not sortTag in ds: return None
        if sortTag=='ImagePositionPatient': z=ds.ImagePositionPatient[2]
        else: z=ds.SliceLocation
        dcmss+=[dict(file=dcm,dataset=ds,z=z)]
    return sorted(dcmss, key=lambda dcms: dcms['z'])

def voxel_array_from_sorted_dicoms(dicomsSorted):
    '''
    extract the 3D voxel array from a list of sorted DICOM objects.
    '''
    ind=0
    
    if len(dicomsSorted) < 1: return None
    ds0=dicomsSorted[0]['dataset']
    
    imwidth,imheight,imdepth=ds0.Rows,ds0.Columns,len(dicomsSorted)
    pixeldata_type=ds0.pixel_array.dtype
    voxels=np.zeros([imwidth,imheight,imdepth],dtype=pixeldata_type)
    
    for i in range(len(dicomsSorted)):
        voxels[:,:,i]=np.transpose(dicomsSorted[i]['dataset'].pixel_array)
        
    return voxels


def get_D(dcm_slice):
    ds=dcm_slice
    try:
        S,X,delta=np.array(ds.ImagePositionPatient),np.array(ds.ImageOrientationPatient),\
            np.array(ds.PixelSpacing)
        if len(S)<3 or len(X)<6 or len(delta)<2:
            raise ValueError('Incorrect orientation tag length')
            
        #Equation C.7.6.2.1-1 (dicom.nema.org)
        D=np.array([[X[0]*delta[0],X[3]*delta[1],0,S[0]],\
                    [X[1]*delta[0],X[4]*delta[1],0,S[1]],\
                    [X[2]*delta[0],X[5]*delta[1],0,S[2]],\
                    [0,0,0,1]])
    except Exception as e:
        print('ERROR: cannot establish correspondence between reference frames due to missing orientation information in DICOM')
        print(e)
        return None    
    return D

def get_dcm_xform(nifti_voxels,dcm_slice0, dcm_slice1):
    nii,ds0,ds1=nifti_voxels,dcm_slice0, dcm_slice1
    try:
        IP,IO,PS=np.array(ds.ImagePositionPatient),np.array(ds.ImageOrientationPatient),\
            np.array(ds.PixelSpacing)
        IO1,IO2=IO[:3],IO[3:]
        R=np.array[IO1,IO2,np.cross(IO1,IO2)]
        M=np.argmax(np.abs(R),0)
        
    
def populate_pixeldata(nifti_voxels, dcm_slice, inverse_qform):
    '''
    populate pixel data in DICOM slice from nifti voxels.
    voxel intensity is determined from scanner-patient coordinates inferred from DICOM and nifti's qform.
    nifti_voxels: Nifti1Image.get_fdata()
    dcm_slice: DicomDataset representing single slice
    inverse_qform: inverse qform (or affine) of Nifti1Image
    '''    
    nii,ds,Qi=nifti_voxels,dcm_slice,inverse_qform
    
    try:
        S,X,delta=np.array(ds.ImagePositionPatient),np.array(ds.ImageOrientationPatient),\
            np.array(ds.PixelSpacing)
        if len(S)<3 or len(X)<6 or len(delta)<2:
            raise ValueError('Incorrect orientation tag length')
            
        #Equation C.7.6.2.1-1 (dicom.nema.org)
        D=np.array([[X[0]*delta[0],X[3]*delta[1],0,S[0]],\
                    [X[1]*delta[0],X[4]*delta[1],0,S[1]],\
                    [X[2]*delta[0],X[5]*delta[1],0,S[2]],\
                    [0,0,0,1]])
        print(D)
    except Exception as e:
        print('ERROR: cannot establish correspondence between reference frames due to missing orientation information in DICOM')
        print(e)
        return False
    
    rows,cols=ds.Rows,ds.Columns
    QiD=np.dot(Qi,D)
    print('Qi',Qi)
    print('D',D)
    print('QiD',QiD)
    zind=np.array([0,0,0,0])
    maxind=np.append(np.array(nii.shape),2)

    slice_voxels=np.zeros([cols,rows]).astype(nii.dtype)
    
    for i,j in it.product(range(rows),range(cols)):
        ind_nii=np.rint(np.dot(QiD,np.array([i,j,0,1]))).astype('int')
        #print("i:{} j:{} nifti_index:{}".format(i,j,ind_nii))
        if (zind<=ind_nii).all() and (ind_nii<maxind).all(): 
            slice_voxels[i,j]=nii[ind_nii[0],ind_nii[1],ind_nii[2]]
 #          print(i,j,nii[ind_nii[0],ind_nii[1],ind_nii[2]],ind_nii)
 #       else: 
 #          print('voxel index outside range!')
 #          print(i,j,ind_nii)
 #          break
        
    ds.PixelData=slice_voxels.tobytes()
    
def compute_flips(affine):
    Q=affine
    F=np.array([[-1,0,0,0],[0,-1,0,0],[0,0,-1,1]])   
#   M=np.argmax(np.abs(Q),0)
    FQ=np.dot(F,Q)
    return np.array([[0,-1*np.sign(Q[M[0],0])],[1,-1*np.sign(Q[M[1],1])],[2,np.sign(Q[M[2],2])]])
    


    

In [3]:
#input variables
input_nifti='/home/shared/NRG/mmilchenko/TMORPH/I3CR/MW101_MR_tx1_GK/test/gk2_struct_Tum1_LFront.nii'
input_dcm='/home/shared/NRG/mmilchenko/TMORPH/I3CR/MW101_MR_tx1_GK/test/study2'
output_dcm='/home/shared/NRG/mmilchenko/TMORPH/I3CR/MW101_MR_tx1_GK/test/gk2_struct_Tum1_LFront_on_study2'

#d='/home/shared/NRG/mmilchenko/REGTEST/TCGA/t1hi/9-AXIAL_T1_PRE-GAD.-05309/'
#input_nifti=d+'nifti2dcm_test/raw.nii'
#input_dcm=d+'raw'
#output_dcm=d+'nifti2dcm_test/raw_nifti2dcm_test'
#d1='1001-POST_COR_CL-55083/'

d1='1001-POST_COR_CL-35509/'
d0='/home/shared/NRG/mmilchenko/REGTEST/TCGA/t1hi/'
input_nifti=d0+d1+'nifti2dcm_test/raw.nii'
input_dcm=d0+d1+'raw'
output_dcm=d0+d1+'nifti2dcm_test/raw_nifti2dcm_test'
#print(d0+d1)


try:
    os.mkdir(output_dcm)
except OSError as error:
    print(error)      

newSeriesInstanceUID='auto'
newSeriesDescription='series generated by nifti2dcm'
newSeriesNumber='2001'

isSC=False #not supported
flipX,flipY,flipZ=False,False,False


[Errno 17] File exists: '/home/shared/NRG/mmilchenko/REGTEST/TCGA/t1hi/1001-POST_COR_CL-35509/nifti2dcm_test/raw_nifti2dcm_test'


In [143]:
#load NIFTI image
#nii=nib.load(input_nifti)
nii=nib.load(input_nifti)

#store to DICOM orientation
nii.header.set_dim_info(None,None,None)
print(nii.get_fdata()[:,:,0].shape)
flips=compute_flips(nii.affine) #np.sign(nii0.affine)
print(flips,'flips')

nii0=nii.as_reoriented(flips)

#nii0=nii.as_reoriented(([[0,1],[1,-1],[2,-1]]))
print(nii0.get_fdata()[:,:,0].shape)


#am=np.argmax(np.abs(nii0.affine),0)
np.set_printoptions(6,suppress=True)

#Q-form, transform from NIFTI coordinates to scanner coordinates
Q=nii.affine
Q1=nii0.affine
Qi=np.linalg.inv(Q)
Q1i=np.linalg.inv(Q1)

print(Q,'Q form')
print(Q1,'Q form reoriented:')
#print('Qi:',Qi)

#print(flips)
#nii.header.set_dim_info(None,None,None)

#print([[0,-1*flips[0,0]],[1,-1*flips[1,1]],[2,flips[2,2]]])
#nii=nii0.as_reoriented([[0,-1*flips[0,0]],[1,-1*flips[1,1]],[2,flips[2,2]]])
#print("axes flips:", [[0,-1*flips[0,0]],[1,-1*flips[1,1]],[2,flips[2,2]]])

dcm_in_files=next(os.walk(input_dcm))[2]
numberOfDicomImages = len(dcm_in_files)
dcm_in_sorted=sort_dcms_by_slice_pos(input_dcm,dcm_in_files,stop_before_pixels=False)

ds0=dcm_in_sorted[0]['dataset']
dsN=dcm_in_sorted[-1]['dataset']

dcm_pixeldata_type=ds0.pixel_array.dtype

#dcm_in_voxels=voxel_array_from_sorted_dicoms(dcm_in_sorted)
#transpose b/c DICOM indexing is (row,col) and NIFTI is (col,row)
v1=nii0.get_fdata().astype(ds0.pixel_array.dtype)
print(v1.shape)
nii_in_voxels=nii.get_fdata().astype(ds0.pixel_array.dtype)
#nii_in_voxels=np.transpose(nii0.get_fdata().astype(ds0.pixel_array.dtype),[1,0,2])
print(nii_in_voxels.shape)

#if dcm_in_voxels.shape != nii_in_voxels.shape:
#    print ('NIFTI and DICOM image shapes don\'t match!')
#    print ('NIFTI shape:',nii_in_voxels.shape)
#    print ('DICOM shape:',dcm_in_voxels.shape)
   

(512, 512)
[[ 0.  1.]
 [ 1. -1.]
 [ 2. -1.]] flips
(512, 512)
[[ -0.46875    0.        -0.       123.039513]
 [ -0.         0.026375  -5.990502  64.402771]
 [  0.         0.468007   0.337599 -99.866776]
 [  0.         0.         0.         1.      ]] Q form
[[ -0.46875    0.         0.       123.039513]
 [  0.        -0.026375   5.990502 -65.891714]
 [  0.        -0.468007  -0.337599 147.387369]
 [  0.         0.         0.         1.      ]] Q form reoriented:
(512, 512, 25)
(512, 512, 25)


In [115]:
IP,IO,PS,SS=np.array(ds0.ImagePositionPatient),np.array(ds0.ImageOrientationPatient),\
            np.array(ds0.PixelSpacing),ds0.SpacingBetweenSlices
IO1,IO2=IO[:3],IO[3:]
R=np.transpose(np.array([IO1,IO2,np.cross(IO1,IO2)]))
M=np.argmax(np.abs(R),0)
print(R,M)
iSL=M[2] #0/1/2 for Sag/Cor/Tra
print(iSL)
VS=np.array([PS[0], PS[1], SS])
print (VS)
np.diag(VS)
R1=np.zeros([4,4])
R1[:3,:3]=np.dot(R,np.diag(VS))
R1[:3,3]=IP
R1[3,3]=1
print(R1)


[[ 1.        0.       -0.      ]
 [ 0.        0.056266  0.998416]
 [ 0.       -0.998416  0.056266]] [0 2 1]
1
[0.46875 0.46875 6.     ]
[[   0.46875     0.          0.       -123.039514]
 [   0.          0.026375    5.990495  -77.880332]
 [   0.         -0.468007    0.337598  139.285009]
 [   0.          0.          0.          1.      ]]


In [153]:
F=np.array([[1, 0, 0, 0],[0,-1,0,511],[0,0,1,0],[0,0,0,1]])
F1=np.array([[-1, 0, 0, 0],[0,-1,0,0],[0,0,1,0],[0,0,0,1]])
#print(F)
#print(F1)
np.dot(Qi,np.dot(np.dot(F1,R1),F))

array([[ 1.      ,  0.      ,  0.      , -0.000002],
       [ 0.      ,  1.      ,  0.      ,  0.000016],
       [ 0.      ,  0.      ,  0.999999, -0.      ],
       [ 0.      ,  0.      ,  0.      ,  1.      ]])

In [142]:
np.dot(Qi,R1)

array([[ -1.      ,   0.      ,   0.      , 524.968589],
       [  0.      ,  -0.993668,   1.438138, 492.303294],
       [  0.      ,  -0.008778,  -0.993667,  25.918953],
       [  0.      ,   0.      ,   0.      ,   1.      ]])

In [134]:
D

array([[   0.46875 ,    0.      ,    0.      , -123.039514],
       [   0.      ,    0.026375,    0.      ,  -77.880332],
       [   0.      ,   -0.468007,    0.      ,  139.285009],
       [   0.      ,    0.      ,    0.      ,    1.      ]])

In [32]:
np.array(dsN.ImagePositionPatient)

array([-123.039514,   65.891549,  147.387369])

In [41]:
Q

array([[ -0.46875 ,   0.      ,  -0.      , 123.039513],
       [ -0.      ,   0.026375,  -5.990502,  64.402771],
       [  0.      ,   0.468007,   0.337599, -99.866776],
       [  0.      ,   0.      ,   0.      ,   1.      ]])

In [150]:
F=np.array([[1, 0, 0, 0],[0,-1,0,512],[0,0,1,0],[0,0,0,1]])
print(F)

[[  1   0   0   0]
 [  0  -1   0 512]
 [  0   0   1   0]
 [  0   0   0   1]]


In [151]:
F1=np.array([[-1, 0, 0, 0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
print(F1)

[[-1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0  1]]


In [152]:
np.dot(Q,F)

array([[ -0.46875 ,   0.      ,   0.      , 123.039513],
       [  0.      ,  -0.026375,  -5.990502,  77.906706],
       [  0.      ,  -0.468007,   0.337599, 139.753006],
       [  0.      ,   0.      ,   0.      ,   1.      ]])

In [44]:
S,X,delta=np.array(ds0.ImagePositionPatient),np.array(ds0.ImageOrientationPatient),\
            np.array(ds0.PixelSpacing)
D=np.array([[X[0]*delta[0],X[3]*delta[1],0,S[0]],\
                    [X[1]*delta[0],X[4]*delta[1],0,S[1]],\
                    [X[2]*delta[0],X[5]*delta[1],0,S[2]],\
                    [0,0,0,1]])
print('S',S)
print('X',X)
print('delta',delta)
print('D',D)

X0=np.array([0,0,0,1])
XX=np.array([1,0,0,1])
XY=np.array([0,1,0,1])
XZ=np.array([0,0,1,1])

print(np.dot(D,X0),'X0')
print(np.dot(D,XX),'XX')
print(np.dot(D,XY),'XY')
print(np.dot(D,XZ),'XZ')


S [-123.039514  -77.880332  139.285009]
X [ 1.        0.        0.        0.        0.056266 -0.998416]
delta [0.46875 0.46875]
D [[   0.46875     0.          0.       -123.039514]
 [   0.          0.026375    0.        -77.880332]
 [   0.         -0.468007    0.        139.285009]
 [   0.          0.          0.          1.      ]]
[-123.039514  -77.880332  139.285009    1.      ] X0
[-122.570764  -77.880332  139.285009    1.      ] XX
[-123.039514  -77.853957  138.817002    1.      ] XY
[-123.039514  -77.880332  139.285009    1.      ] XZ


In [6]:
S1,X1,delta=np.array(dsN.ImagePositionPatient),np.array(dsN.ImageOrientationPatient),\
            np.array(dsN.PixelSpacing)
D1=np.array([[X1[0]*delta[0],X1[3]*delta[1],0,S1[0]],\
                    [X1[1]*delta[0],X1[4]*delta[1],0,S1[1]],\
                    [X1[2]*delta[0],X1[5]*delta[1],0,S1[2]],\
                    [0,0,0,1]])
print('S1',S1)
print('X1',X1)
print('delta',delta)
print('D1',D1)

S1 [-123.0395   65.8915  147.3874]
X1 [ 1.      0.      0.      0.      0.0563 -0.9984]
delta [0.4688 0.4688]
D1 [[   0.4688    0.        0.     -123.0395]
 [   0.        0.0264    0.       65.8915]
 [   0.       -0.468     0.      147.3874]
 [   0.        0.        0.        1.    ]]


In [7]:
Q1iD=np.dot(Q1i,D)
print(Q1iD)

[[ -1.       0.       0.     524.9686]
 [  0.       0.9937   0.      18.6967]
 [  0.       0.0088   0.      -1.919 ]
 [  0.       0.       0.       1.    ]]


In [ ]:
np.dot(Q1iD,X0)

In [ ]:
Q1iD1=np.dot(Q1i,D1)
print(Q1iD1)
np.dot(Q1iD1,X0)

In [149]:
flipX,flipY,flipZ=False,True,False
if flipX: nii_in_voxels=np.flip(nii_in_voxels,0)
if flipY: nii_in_voxels=np.flip(nii_in_voxels,1)
if flipZ: nii_in_voxels=np.flip(nii_in_voxels,2)

siUID=pydicom.uid.generate_uid() if newSeriesInstanceUID == 'auto' else newSeriesInstanceUID
sDescr=ds.SeriesDescription if newSeriesDescription is None else newSeriesDescription
sNumber=ds.SeriesNumber if newSeriesNumber is None else newSeriesNumber

#cycle through input DICOM datasets and replace voxels.
for i in range(len(dcm_in_sorted)):
    ds=dcm_in_sorted[i]['dataset']
    ds.PixelData=np.transpose(nii_in_voxels[:,:,i]).tobytes()
    #populate_pixeldata(nii_in_voxels, ds, Q1i)
    ds[0x0020, 0x000e].value=siUID
    ds[0x0008,0x103e].value=sDescr
    ds[0x0020, 0x0011].value=sNumber    
    ds.save_as(output_dcm+'/'+str(i)+'.dcm')
    

In [ ]:
flipZ


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.imshow(ds.pixel_array, interpolation='nearest')
